In [8]:
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langserve import RemoteRunnable
import bs4
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from typing import Tuple, List
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools.render import render_text_description_and_args
from langchain_core.messages import AIMessage, HumanMessage
from langchain.agents.format_scratchpad import format_log_to_messages
from langchain.agents.output_parsers import (
    ReActJsonSingleInputOutputParser,
)
from langchain.pydantic_v1 import BaseModel, Field
from langchain.agents import AgentExecutor
from langchain_community.chat_models import ChatOllama
from langchain_openai import ChatOpenAI

In [9]:
# import os

# os.environ["OPENAI_API_KEY"] = "your key" # OpenAI API KEY (https://platform.openai.com/api-keys)
# os.environ["TAVILY_API_KEY"] = "your key" # Tavily Search API KEY (https://app.tavily.com/home)

from dotenv import load_dotenv

# 환경변수 로드 (.env)
load_dotenv()

True

In [5]:
from langchain_community.tools.tavily_search import TavilySearchResults

# Tavily Search API : 효율적이고 빠른 검색 결과를 목표로 하는 LLM 및 RAG에 최적화된 검색 엔진
tavily_search = TavilySearchResults(k=3)
tavily_search.invoke("0koang velog 주소")

[{'url': 'https://velog.io/@kwon0koang/series/DevOps',
  'content': '개발자들을 위한 블로그 서비스. 어디서 글 쓸지 고민하지 말고 벨로그에서 시작하세요.'},
 {'url': 'https://www.youtube.com/watch?v=M8ptbQ8EgtU',
  'content': "grab bbq pork rice with us! we were inspired to visit the city of darkness after watching the new hong kong movie 'twilight of the warriors: walled in'. the..."},
 {'url': 'https://alpha.velog.io/@kwon0koang/posts',
  'content': 'kwon0koang님이 작성한 포스트 시리즈들을 확인해보세요. 서비스 핵심 가치를 이해하고, 지속적인 개선을 이끄는 엔지니어(를 지향함)'},
 {'url': 'https://velog.io/@kwon0koang/series',
  'content': '0koang. 서비스 핵심 가치를 이해하고, 지속적인 개선을 이끄는 엔지니어 (를 지향함) 7 팔로워 2 팔로잉. 팔로우. 글 시리즈 소개. kwon0koang님이 작성한 포스트 시리즈들을 확인해보세요.'},
 {'url': 'https://alpha.velog.io/@kwon0koang/about',
  'content': 'kwon0koang님의 자기소개가 비었습니다. 서비스 핵심 가치를 이해하고, 지속적인 개선을 이끄는 엔지니어(를 지향함)'}]

In [6]:
from langchain_openai import ChatOpenAI
from langchain_google_vertexai import ChatVertexAI
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model="EEVE-Korean-Instruct-10.8B-v1.0:latest", temperature=0)
# llm = ChatOpenAI(model="EEVE-Korean-Instruct-10.8B-v1.0:latest", temperature=0)
# llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
# chat = ChatVertexAI(model="gemini-pro", convert_system_message_to_human=True)

In [23]:
prompt = ChatPromptTemplate.from_messages([
    ("system", """
너는 유능한 업무 보조자야.
다음 context를 사용해서 question에 대한 답을 말해줘.
정답을 모르면 모른다고만 해.

<question>
{question}
</question>

<context>
{context}
</context>

# answer :
"""
    ),
])

retrieved_docs = []
def get_page_contents_with_metadata(docs) -> str: 
    global retrieved_docs
    retrieved_docs = docs
    
    result = ""
    for i, doc in enumerate(docs):
        if i > 0:
            result += "\n\n"
        result += f"## 본문: {doc['content']}\n### 출처: {doc['url']}"
    
    return result

def get_metadata_sources(docs) -> str: 
    sources = set()
    for doc in docs:
        sources.add(doc['url'])
    return "\n".join(sources)

def parse(ai_message: AIMessage) -> str:
    """Parse the AI message and add source."""
    return f"{ai_message.content}\n\n[출처]\n{get_metadata_sources(retrieved_docs)}"

chain = (
    {"context": tavily_search | get_page_contents_with_metadata, "question": RunnablePassthrough()}
    | prompt
    | llm
    | parse
)

result = chain.invoke("0koang velog 주소?")
print(f"{result}")

0koang의 베로그 주소는 다음과 같습니다: https://velog.io/@kwon0koang

[출처]
https://www.youtube.com/watch?v=HTi3EIxl5Pg
https://velog.io/@kwon0koang/series
https://alpha.velog.io/@kwon0koang/about
https://www.youtube.com/watch?v=M8ptbQ8EgtU
https://alpha.velog.io/@kwon0koang/posts
